### Building a Better (and More Realistic) Momentum Strategy
Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:
* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
    - Suppose A stock show consistant perform even small (0.5% or 1%) consider hqm 
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.
    - A stock which shows sudden or some month higher return (e.g. 10%) and remaning month lower 
----
The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by `short-term news` that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).
To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentile
* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation hqm often. It stands for high-quality-momentum stocks.



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf 
import talib

In [5]:
from dataclasses import dataclass


@dataclass
class HQM:
  df: pd.DataFrame
  portfolio_value: float = 100000


  def get_stock_stats(self) -> pd.DataFrame:

    position_size = self.portfolio_value / len(self.df.columns)
    # floor division operator - to get the whole number of share
    number_of_share_to_buy = (position_size // self.df.iloc[-1]).astype('int64')

    changes = {}
    
    periods = {
        'year5': 252 * 5,
        'year3': 252 * 3,
        'year2': 252 * 2,
        'year1': 252,
        'month6': 21 * 6,
        'month3': 21 * 3,
        'month1': 21,
        'day30': 30,
        'day5': 5

    }
    
    max_value = self.df.max()
    min_value = self.df.min()
    latest_price = self.df.iloc[-1]

    changes['maxChanges'] = ((max_value - min_value ) / min_value) * 100
    
    for period_name, period_number in periods.items():
      # Validating if the datapoint are available for proper calculation
      if period_number >= len(self.df):
        continue
      past_price = self.df.iloc[-period_number]
      changes[f"{period_name}ChnagePercentage"] = (latest_price - past_price) / past_price * 100

    dataframe = pd.DataFrame.from_dict(changes)
    dataframe['latest Price'] = latest_price
    dataframe['Number of Share to buy'] = number_of_share_to_buy
    
    return dataframe


  def get_momentum_data(self) -> pd.DataFrame:
    df  = self.get_stock_stats()

    # Creating the empty dataframe
    new_df = pd.DataFrame()
    new_df_col = ['latest Price','Number of Share to buy' ,'year1ChnagePercentage','month6ChnagePercentage', 'month3ChnagePercentage', 'month1ChnagePercentage']


    for col in new_df_col:
      if col in df.columns:
        new_df[col] = df[col]
        if col not in ['latest Price', 'Number of Share to buy']:
          new_df[f'{col}Percentaile'] = np.nan
      else:
        new_df[col] = np.nan

    update_col = ['latest Price', 'Number of Share to buy', 'One Year Return', 'One Year Return Percentaile' ,'Six Month Return', 
                  'Six Month Return Percentaile','Three Month Return', 'Three Month Return Percentaile', 
                  'One Month Return', 'One Month Return Percentaile']
    
    new_df.columns = update_col  
    return new_df


In [6]:
ticker_list =  ['AAPL', 'MSFT', 'GOOG', 'AMZN', 'TSLA', 'JPM', 'BAC', 'V', 'MA', 'PG', 'KO', 'WMT', 'PEP', 'JNJ', 'UNH', 'PFE', 'ABBV', 'LLY']
ticker_df = yf.download(ticker_list, start='2020-01-01', end='2023-12-31', interval='1D')['Adj Close']

[*********************100%%**********************]  18 of 18 completed


In [7]:
clsobj = HQM(ticker_df).get_momentum_data()
clsobj

,latest Price,Number of Share to buy,One Year Return,One Year Return Percentaile,Six Month Return,Six Month Return Percentaile,Three Month Return,Three Month Return Percentaile,One Month Return,One Month Return Percentaile
Ticker,,,,,,,,,,
AAPL,192.024185,28,49.375937,NaN,0.303417,NaN,10.954616,NaN,1.358255,NaN
ABBV,154.970001,35,-4.669043,NaN,14.784096,NaN,4.532884,NaN,8.834891,NaN
AMZN,151.940002,36,80.494181,NaN,16.679466,NaN,17.364433,NaN,4.004385,NaN
BAC,33.669998,165,1.599272,NaN,15.308210,NaN,26.104858,NaN,10.429644,NaN
GOOG,140.929993,39,58.437322,NaN,16.896148,NaN,4.261297,NaN,5.234464,NaN
JNJ,156.740005,35,-11.725610,NaN,-4.046526,NaN,1.024822,NaN,1.344887,NaN
JPM,170.100006,32,27.683534,NaN,16.022103,NaN,18.313974,NaN,8.982576,NaN
KO,58.930000,94,-7.849883,NaN,-2.723674,NaN,6.218459,NaN,0.838470,NaN
LLY,582.919983,9,58.825132,NaN,26.331751,NaN,8.291071,NaN,-1.373849,NaN
